## Moirai version 3

In [65]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns


In [66]:
# Import the data 
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [67]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data.csv')

# Reset the index
df = df.reset_index()

In [68]:
# Convert 'date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

# Ensure the DataFrame is sorted by the index
df.sort_index(inplace=True)

# setting daily frequency
df = df.asfreq('D')


In [69]:
# Create column called item_id with only 0 
df['item_id'] = 0

In [70]:
df




,Day_ahead_price (€/MWh),Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh),item_id
Date,,,,,,,,,,,,,,,,,,,,
2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.1000,250.979,0
2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.0000,258.671,0
2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.9000,271.495,0
2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.4000,270.613,0
2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.3000,287.555,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-25,61.09,97.58,3.43,4.52,123.683,48.501,58.671,193.697,255.124,60.627,13.565,169.216,29.879,278.787,-36.930,947,75.22,23.7625,0.000,0
2024-02-26,66.27,73.25,3.12,4.96,124.810,69.146,58.444,281.177,289.764,59.810,12.231,110.504,62.336,239.555,-198.686,947,75.09,23.9000,0.000,0
2024-02-27,73.84,58.12,3.11,4.53,124.989,103.379,59.181,351.355,354.042,67.170,23.753,85.584,16.951,131.761,-209.332,947,76.11,24.8300,0.000,0


In [71]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from gluonts.dataset.pandas import PandasDataset
from huggingface_hub import hf_hub_download
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_single
from gluonts.dataset.split import split


# Assume 'your_data_with_covariates.csv' has columns: ['date', 'target', 'covariate1', 'covariate2', ...]

# Step 2: Split by Date
split_date = "2022-07-01"
train_df = df.loc[:split_date]
test_df = df.loc[split_date:]

# Step 3: Specify Target and Covariates
# 'target' is the column you're predicting; other columns are the covariates
feat_dynamic_real = ['Solar_radiation (W/m2)', 'Wind_speed (m/s)', 'Temperature (°C)', 'Biomass (GWh)',
                                 'Hard_coal (GWh)', 'Hydro (GWh)', 'Lignite (GWh)', 'Natural_gas (GWh)', 'Other (GWh)',
                                 'Pumped_storage_generation (GWh)', 'Solar_energy (GWh)', 'Wind_offshore (GWh)',
                                 'Wind_onshore (GWh)', 'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)',
                                 'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)']  

# Convert to PandasDataset including covariates
train_dataset = PandasDataset.from_long_dataframe(
    df=train_df,
    target='Day_ahead_price (€/MWh)',
    item_id='item_id',  
    feat_dynamic_real=feat_dynamic_real  # Your covariates
)

test_dataset = PandasDataset.from_long_dataframe(
    df=test_df,
    target='Day_ahead_price (€/MWh)',
    item_id='item_id',
    feat_dynamic_real=feat_dynamic_real
)

# Step 4: Download Pre-trained Moirai Model
SIZE = "small"  # Choose from {'small', 'base', 'large'}
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}")
)

# Step 5: Fine-tuning the Pre-trained Model with Covariates
PDT = 20  # Prediction length
CTX = 200  # Context length
BSZ = 32  # Batch size

# Train the model on the train dataset with covariates (for fine-tuning)
predictor = model.create_predictor(batch_size=BSZ)

# Step 6: Making Predictions
forecasts = predictor.predict(test_dataset.input)

# Step 7: Plot Predictions vs Actual Data
input_it = iter(test_dataset.input)
label_it = iter(test_dataset.label)
forecast_it = iter(forecasts)

inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

plot_single(
    inp, 
    label, 
    forecast, 
    context_length=CTX,
    name="Moirai Prediction with Covariates",
    show_label=True
)
plt.show()


TypeError: PandasDataset.from_long_dataframe() missing 1 required positional argument: 'dataframe'

In [32]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch.batchify import batchify
from torch.utils.data import DataLoader
from huggingface_hub import hf_hub_download
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_single

# Load and prepare the data
def load_and_prepare_data(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df = df.asfreq('D')  # Ensure daily frequency
    return df

# Load the dataset
df = load_and_prepare_data('../../data/Final_data/final_data.csv')

# Adding an 'item_id' column with a constant value (for a univariate dataset)
df['item_id'] = 0

# Specify the target column and covariates
target_column = 'Day_ahead_price (€/MWh)'   # Replace with your actual target column name
covariate_columns = ['Solar_radiation (W/m2)', 'Wind_speed (m/s)', 'Temperature (°C)', 'Biomass (GWh)',
                     'Hard_coal (GWh)', 'Hydro (GWh)', 'Lignite (GWh)', 'Natural_gas (GWh)', 'Other (GWh)',
                     'Pumped_storage_generation (GWh)', 'Solar_energy (GWh)', 'Wind_offshore (GWh)',
                     'Wind_onshore (GWh)', 'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)',
                     'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)']

# Split by Date
split_date = '2022-07-01'
train_df = df.loc[:split_date]
test_df = df.loc[split_date:]

# Ensure the target and covariate columns are in the DataFrame for both train and test datasets
train_df = train_df[['item_id', target_column] + covariate_columns]
test_df = test_df[['item_id', target_column] + covariate_columns]

# Convert to PandasDataset with covariates
train_dataset = PandasDataset.from_long_dataframe(
    dataframe=train_df,
    target=target_column,
    item_id='item_id',
    feat_dynamic_real=covariate_columns
)

test_dataset = PandasDataset.from_long_dataframe(
    dataframe=test_df,
    target=target_column,
    item_id='item_id',
    feat_dynamic_real=covariate_columns
)

# Step 4: Download Pre-trained Moirai Model
SIZE = "small"  # Choose from {'small', 'base', 'large'}
PDT = 20  # Prediction length
CTX = 200  # Context length
BSZ = 32  # Batch size

# Initialize the model with all required parameters
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,  # Set the prediction length
    target_dim=1,  # Target dimension is 1 for univariate forecasting
    feat_dynamic_real_dim=len(covariate_columns),  # Number of dynamic covariates
    past_feat_dynamic_real_dim=len(covariate_columns),  # Set to the same if past covariates exist
    context_length=CTX  # Set the context length
)

# Step 5: DataLoader Setup
from torch.utils.data import DataLoader

# Step 5: DataLoader Setup
train_loader = DataLoader(train_dataset, batch_size=BSZ, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BSZ)



# Step 6: Fine-tuning the Pre-trained Model with Covariates
predictor = model.create_predictor(batch_size=BSZ)

# Step 7: Making Predictions using DataLoader
for batch in test_loader:
    forecasts = predictor.predict(batch)
    # You can access forecast results here
    # Example plotting for one forecast
    plot_single(
        batch['past_target'],  # Input past target data
        batch['future_target'],  # Actual future target data
        forecasts,  # Predictions from the model
        context_length=CTX,
        name="Moirai Prediction with Covariates",
        show_label=True
    )
    plt.show()


TypeError: 'PandasDataset' object is not subscriptable

In [48]:
from gluonts.dataset.common import ListDataset

# Step 3: Specify Target and Covariates
target_column = 'Day_ahead_price (€/MWh)'  # Replace with your actual target column name
covariate_columns = ['Solar_radiation (W/m2)', 'Wind_speed (m/s)', 'Temperature (°C)', 'Biomass (GWh)',
                     'Hard_coal (GWh)', 'Hydro (GWh)', 'Lignite (GWh)', 'Natural_gas (GWh)', 'Other (GWh)',
                     'Pumped_storage_generation (GWh)', 'Solar_energy (GWh)', 'Wind_offshore (GWh)',
                     'Wind_onshore (GWh)', 'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)',
                     'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)']


# Split by Date
split_date_1 = '2022-07-01'
split_data_2 = '2022-06-30'
train_df = df.loc[:split_data_2]
test_df = df.loc[split_date_1:]

# Create a ListDataset for train and test datasets
train_dataset = ListDataset(
    [
        {
            "start": train_df.index[0],  # First date in the dataset
            "target": train_df[target_column].values,  # Target values for training
            "feat_dynamic_real": train_df[covariate_columns].T.values  # Covariates for training
        }
    ],
    freq='D'  # Adjust frequency if different
)

test_dataset = ListDataset(
    [
        {
            "start": test_df.index[0],  # First date in the dataset
            "target": test_df[target_column].values,  # Target values for testing
            "feat_dynamic_real": test_df[covariate_columns].T.values  # Covariates for testing
        }
    ],
    freq='D'
)


In [49]:
from gluonts.dataset.loader import TrainDataLoader, InferenceDataLoader
from gluonts.torch.batchify import batchify

# Create the GluonTS data loaders
train_loader = TrainDataLoader(train_dataset, batch_size=BSZ, stack_fn=batchify)
test_loader = InferenceDataLoader(test_dataset, batch_size=BSZ, stack_fn=batchify)

In [53]:
# Step 1: Create properly formatted dataset
train_dataset = ListDataset(
    [{
        "start": train_df.index[0],  # First date in your training dataset
        "target": train_df[target_column].values,  # Target values
        "feat_dynamic_real": train_df[covariate_columns].T.values  # Dynamic covariates
    }],
    freq="D"  # Daily frequency
)

test_dataset = ListDataset(
    [{
        "start": test_df.index[0],  # First date in your test dataset
        "target": test_df[target_column].values,
        "feat_dynamic_real": test_df[covariate_columns].T.values
    }],
    freq="D"
)

# Step 2: Ensure correct DataLoader
train_loader = TrainDataLoader(train_dataset, batch_size=BSZ, stack_fn=batchify)
test_loader = InferenceDataLoader(test_dataset, batch_size=BSZ, stack_fn=batchify)

# Step 3: Inspect batch structure to confirm it's correctly formatted
for batch in test_loader:
    print(batch)  # Inspect to ensure the structure is correct
    
    # Step 4: Try making predictions
    forecasts = list(predictor.predict(batch))  # Convert generator to list

    # Convert target arrays to numpy for plotting
    inp_dict = {"target": batch["target"].cpu().numpy(), "start": batch["start"]}

    # Plotting example
    forecasts_np = np.array(forecasts).squeeze()  # Ensure proper dimensions
    plot_single(
        inp_dict,
        {"target": forecasts_np, "start": batch["start"]},  # Ensure same structure
        forecasts_np,
        context_length=CTX,
        name="Moirai Prediction with Covariates",
        show_label=True
    )
    plt.show()


{'start': [Period('2022-07-01', 'D')], 'target': tensor([[ 3.1438e+02,  2.1892e+02,  2.0011e+02,  2.9389e+02,  3.1837e+02,
          3.0354e+02,  2.1322e+02,  2.9592e+02,  1.4814e+02,  1.1516e+02,
          3.7260e+02,  3.7766e+02,  3.5216e+02,  3.6738e+02,  3.2992e+02,
          1.7481e+02,  2.2186e+02,  3.9607e+02,  3.9709e+02,  3.3270e+02,
          3.9673e+02,  3.7745e+02,  3.2499e+02,  2.6063e+02,  3.0981e+02,
          3.1657e+02,  4.2449e+02,  4.7325e+02,  4.3865e+02,  3.8077e+02,
          3.1779e+02,  4.2462e+02,  3.8051e+02,  3.8483e+02,  4.1416e+02,
          3.8031e+02,  2.7381e+02,  2.1492e+02,  3.5955e+02,  3.4487e+02,
          3.3508e+02,  3.6815e+02,  4.3680e+02,  3.7774e+02,  3.3046e+02,
          4.2660e+02,  4.9079e+02,  5.5195e+02,  5.6376e+02,  5.1510e+02,
          4.5797e+02,  3.6413e+02,  5.6381e+02,  6.0513e+02,  6.2434e+02,
          5.9983e+02,  6.9944e+02,  6.0784e+02,  4.0103e+02,  6.5841e+02,
          6.6024e+02,  6.0449e+02,  5.7120e+02,  4.2225e+02,  2

AttributeError: 'str' object has no attribute 'copy'

In [54]:
for batch in test_loader:
    print(batch)  # Check if there are any string values or unexpected structures


{'start': [Period('2022-07-01', 'D')], 'target': tensor([[ 3.1438e+02,  2.1892e+02,  2.0011e+02,  2.9389e+02,  3.1837e+02,
          3.0354e+02,  2.1322e+02,  2.9592e+02,  1.4814e+02,  1.1516e+02,
          3.7260e+02,  3.7766e+02,  3.5216e+02,  3.6738e+02,  3.2992e+02,
          1.7481e+02,  2.2186e+02,  3.9607e+02,  3.9709e+02,  3.3270e+02,
          3.9673e+02,  3.7745e+02,  3.2499e+02,  2.6063e+02,  3.0981e+02,
          3.1657e+02,  4.2449e+02,  4.7325e+02,  4.3865e+02,  3.8077e+02,
          3.1779e+02,  4.2462e+02,  3.8051e+02,  3.8483e+02,  4.1416e+02,
          3.8031e+02,  2.7381e+02,  2.1492e+02,  3.5955e+02,  3.4487e+02,
          3.3508e+02,  3.6815e+02,  4.3680e+02,  3.7774e+02,  3.3046e+02,
          4.2660e+02,  4.9079e+02,  5.5195e+02,  5.6376e+02,  5.1510e+02,
          4.5797e+02,  3.6413e+02,  5.6381e+02,  6.0513e+02,  6.2434e+02,
          5.9983e+02,  6.9944e+02,  6.0784e+02,  4.0103e+02,  6.5841e+02,
          6.6024e+02,  6.0449e+02,  5.7120e+02,  4.2225e+02,  2

In [55]:
for entry in train_dataset:
    if isinstance(entry["target"], str) or isinstance(entry["feat_dynamic_real"], str):
        raise ValueError("Found string where numeric data was expected")


In [56]:
train_dataset = ListDataset(
    [{
        "start": pd.Period('2022-07-01', 'D'),  # Ensure correct period
        "target": train_df[target_column].values.tolist(),  # Ensure target is a list
        "feat_dynamic_real": train_df[covariate_columns].T.values.tolist()  # Dynamic covariates as a list of lists
    }],
    freq="D"
)


In [60]:
from gluonts.dataset.common import ListDataset
import numpy as np
import pandas as pd

# Assuming train_df and test_df are Pandas DataFrames
train_dataset = ListDataset(
    [{
        "start": pd.Timestamp(train_df.index[0]),  # Ensure correct timestamp
        "target": train_df[target_column].values.tolist(),  # Convert target to a list
        "feat_dynamic_real": train_df[covariate_columns].T.values.tolist()  # Convert dynamic covariates to a list of lists
    }],
    freq="D"
)

test_dataset = ListDataset(
    [{
        "start": pd.Timestamp(test_df.index[0]),  # Ensure correct timestamp
        "target": test_df[target_column].values.tolist(),
        "feat_dynamic_real": test_df[covariate_columns].T.values.tolist()
    }],
    freq="D"
)

for entry in train_dataset:
    print(entry)  # Ensure that "target", "feat_dynamic_real" are lists or arrays, not strings


{'start': Period('2012-01-01', 'D'), 'target': array([ 18.19,  33.82,  35.03, ..., 331.52, 315.54, 325.48], dtype=float32), 'feat_dynamic_real': array([[ 14.75 ,  15.12 ,  31.88 , ..., 271.71 , 219.42 , 272.83 ],
       [  4.95 ,   5.   ,   7.77 , ...,   2.66 ,   2.61 ,   2.7  ],
       [  8.39 ,   7.41 ,   5.23 , ...,  18.55 ,  19.88 ,  21.67 ],
       ...,
       [ 99.64 , 100.04 , 100.44 , ..., 107.27 , 106.79 , 105.88 ],
       [ 21.1  ,  20.   ,  20.9  , ..., 130.   , 140.55 , 149.75 ],
       [250.979, 258.671, 271.495, ...,  95.041,  94.657,  94.343]],
      dtype=float32)}


In [61]:
for batch in test_loader:
    print(batch)  # Inspect the batch for correct structure
    forecasts = list(predictor.predict(batch))  # Perform prediction


{'start': [Period('2022-07-01', 'D')], 'target': tensor([[ 3.1438e+02,  2.1892e+02,  2.0011e+02,  2.9389e+02,  3.1837e+02,
          3.0354e+02,  2.1322e+02,  2.9592e+02,  1.4814e+02,  1.1516e+02,
          3.7260e+02,  3.7766e+02,  3.5216e+02,  3.6738e+02,  3.2992e+02,
          1.7481e+02,  2.2186e+02,  3.9607e+02,  3.9709e+02,  3.3270e+02,
          3.9673e+02,  3.7745e+02,  3.2499e+02,  2.6063e+02,  3.0981e+02,
          3.1657e+02,  4.2449e+02,  4.7325e+02,  4.3865e+02,  3.8077e+02,
          3.1779e+02,  4.2462e+02,  3.8051e+02,  3.8483e+02,  4.1416e+02,
          3.8031e+02,  2.7381e+02,  2.1492e+02,  3.5955e+02,  3.4487e+02,
          3.3508e+02,  3.6815e+02,  4.3680e+02,  3.7774e+02,  3.3046e+02,
          4.2660e+02,  4.9079e+02,  5.5195e+02,  5.6376e+02,  5.1510e+02,
          4.5797e+02,  3.6413e+02,  5.6381e+02,  6.0513e+02,  6.2434e+02,
          5.9983e+02,  6.9944e+02,  6.0784e+02,  4.0103e+02,  6.5841e+02,
          6.6024e+02,  6.0449e+02,  5.7120e+02,  4.2225e+02,  2

AttributeError: 'str' object has no attribute 'copy'

In [63]:
from gluonts.dataset.common import ListDataset

# Convert train and test datasets to ListDataset for Moirai model
train_dataset = ListDataset(
    [{
        'start': train_df.index[0],
        'target': train_df['Day_ahead_price (€/MWh)'].values,
        'feat_dynamic_real': train_df[columns_to_scale].T.values
    }],
    freq='D'
)

test_dataset = ListDataset(
    [{
        'start': test_df.index[0],
        'target': test_df['Day_ahead_price (€/MWh)'].values,
        'feat_dynamic_real': test_df[columns_to_scale].T.values
    }],
    freq='D'
)

# Initialize the Moirai model and the predictor
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=len(columns_to_scale),
    past_feat_dynamic_real_dim=len(columns_to_scale),
)

# Create predictor
predictor = model.create_predictor(batch_size=BSZ)

# Make predictions using the test dataset
forecasts = list(predictor.predict(test_dataset))

# Prepare data for plotting
input_it = iter(test_dataset)
label_it = iter(test_dataset)
forecast_it = iter(forecasts)

inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

# Convert tensors to NumPy arrays
forecast_values = forecast.mean.numpy() if torch.is_tensor(forecast.mean) else forecast.mean
label_values = label['target'].numpy() if torch.is_tensor(label['target']) else label['target']

# Reshape and inverse scale the predictions and labels
forecast_values = forecast_values.reshape(-1, 1)
label_values = label_values.reshape(-1, 1)
forecast_values_original_scale = target_scaler.inverse_transform(forecast_values).flatten()
label_values_original_scale = target_scaler.inverse_transform(label_values).flatten()

# Plotting the forecast vs actual values
start_date = inp['start'].to_timestamp() if hasattr(inp['start'], 'to_timestamp') else pd.Timestamp(inp['start'])
test_start_date = pd.Timestamp(start_date) + pd.Timedelta(days=len(inp['target']))
test_dates = pd.date_range(start=test_start_date, periods=PDT, freq='D')

forecast_trace = go.Scatter(
    x=test_dates,
    y=forecast_values_original_scale,
    mode='lines',
    name='Forecasted Data',
    line=dict(color='darkred', dash='solid')
)

label_trace = go.Scatter(
    x=test_dates,
    y=label_values_original_scale,
    mode='lines',
    name='Actual Test Data',
    line=dict(color='darkgreen', dash='solid')
)

fig = go.Figure(data=[forecast_trace, label_trace])
fig.show()


KeyError: 'past_feat_dynamic_real'

In [64]:
from gluonts.dataset.common import ListDataset

# Convert train and test datasets to ListDataset for Moirai model
train_dataset = ListDataset(
    [{
        'start': train_df.index[0],
        'target': train_df['Day_ahead_price (€/MWh)'].values,
        'feat_dynamic_real': train_df[columns_to_scale].T.values
    }],
    freq='D'
)

test_dataset = ListDataset(
    [{
        'start': test_df.index[0],
        'target': test_df['Day_ahead_price (€/MWh)'].values,
        'feat_dynamic_real': test_df[columns_to_scale].T.values
    }],
    freq='D'
)

# Initialize the Moirai model and the predictor
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=len(columns_to_scale),
    past_feat_dynamic_real_dim=len(columns_to_scale),
)

# Create predictor
predictor = model.create_predictor(batch_size=BSZ)

# Make predictions using the test dataset
forecasts = list(predictor.predict(test_dataset))

# Prepare data for plotting
input_it = iter(test_dataset)
label_it = iter(test_dataset)
forecast_it = iter(forecasts)

inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

# Convert tensors to NumPy arrays
forecast_values = forecast.mean.numpy() if torch.is_tensor(forecast.mean) else forecast.mean
label_values = label['target'].numpy() if torch.is_tensor(label['target']) else label['target']

# Reshape and inverse scale the predictions and labels
forecast_values = forecast_values.reshape(-1, 1)
label_values = label_values.reshape(-1, 1)
forecast_values_original_scale = target_scaler.inverse_transform(forecast_values).flatten()
label_values_original_scale = target_scaler.inverse_transform(label_values).flatten()

# Plotting the forecast vs actual values
start_date = inp['start'].to_timestamp() if hasattr(inp['start'], 'to_timestamp') else pd.Timestamp(inp['start'])
test_start_date = pd.Timestamp(start_date) + pd.Timedelta(days=len(inp['target']))
test_dates = pd.date_range(start=test_start_date, periods=PDT, freq='D')

forecast_trace = go.Scatter(
    x=test_dates,
    y=forecast_values_original_scale,
    mode='lines',
    name='Forecasted Data',
    line=dict(color='darkred', dash='solid')
)

label_trace = go.Scatter(
    x=test_dates,
    y=label_values_original_scale,
    mode='lines',
    name='Actual Test Data',
    line=dict(color='darkgreen', dash='solid')
)

fig = go.Figure(data=[forecast_trace, label_trace])
fig.show()


KeyError: 'past_feat_dynamic_real'